<a href="https://colab.research.google.com/github/pnkjjha/BuildNExperiment/blob/main/Structured_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark[sql]

In [2]:
from pyspark.sql import SparkSession
from pandas import json_normalize
import pyspark.sql.functions as F
import pyspark.sql.types as T
import random
import requests
import json
import os
import pandas as pd
import datetime

In [ ]:
# spark=SparkSession.builder.appName('test').config("spark.sql.shuffle.partitions", "4").getOrCreate()

In [3]:
spark=SparkSession.builder.appName('test').getOrCreate()

In [ ]:
# url = "https://api.carbonintensity.org.uk/intensity"

# while switch:
#   response = requests.get(url)
#   json_normalize(response.json())
#   message = response.json()
#   json_normalize(message['data']).rename(columns={"intensity.forecast": "intensity_forecast",\
#                                                   "intensity.actual": "intensity_actual",\
#                                                   "intensity.percentile": "intensity_percentile",\
#                                                   "intensity.index": "intensity_index"\
#                                                   }).to_csv("intensity_"+str(random.randint(1,10000))+".csv", index=False)


#Inputfile generation

In [4]:
%rm -rf input
%mkdir input

In [ ]:
datetime.datetime.strftime(datetime.datetime.utcnow(),"%Y-%m-%d %H:%M:%S")

'2024-09-16 06:57:36'

In [14]:
i = 0
s = requests.Session()
url = "https://stream.wikimedia.org/v2/stream/recentchange"
file_number = str(random.randint(1,10000))
with s.get(url, stream=True) as resp:
    for line in resp.iter_lines(1024,decode_unicode=True):
        # Check if line has some bytes in it
        if line:
            # convert line into json
            # get data lines only
            if line[:4] == "data":
                # print(line[5:])
                json_line = json.loads(line[5:])
                flatlines = pd.json_normalize(json_line,sep="_")
                with open("input/rc"+str(random.randint(1,10000))+".json","a",encoding="utf-8") as f:
                    flatlines.to_json(f,orient="records",lines=True,index=False)
                    i = i + 1
                    if i>=4 :
                        break

In [6]:
%ls -l input | wc -l

5


In [7]:
# design schema here
dSchema = T.StructType([\
   T.StructField("$Schema", T.StringType())\
  ,T.StructField("id", T.IntegerType())\
  ,T.StructField("type", T.StringType())\
  ,T.StructField("namespace", T.IntegerType())\
  ,T.StructField("title", T.StringType())\
  ,T.StructField("title_url", T.StringType())\
  ,T.StructField("comment", T.StringType())\
  ,T.StructField("timestamp", T.IntegerType())\
  ,T.StructField("user", T.StringType())\
  ,T.StructField("bot", T.BooleanType())\
  ,T.StructField("log_id", T.IntegerType())\
  ,T.StructField("log_type", T.StringType())\
  ,T.StructField("log_action", T.StringType())\
  ,T.StructField("log_action_comment", T.StringType())\
  ,T.StructField("server_url", T.StringType())\
  ,T.StructField("server_name", T.StringType())\
  ,T.StructField("server_script_path", T.StringType())\
  ,T.StructField("wiki", T.StringType())\
  ,T.StructField("parsedcomment", T.StringType())\
  ,T.StructField("meta_uri", T.StringType())\
  ,T.StructField("meta_request_id", T.StringType())\
  ,T.StructField("meta_id", T.StringType())\
  ,T.StructField("meta_dt", T.TimestampType())\
  ,T.StructField("meta_domain", T.StringType())\
  ,T.StructField("meta_stream", T.StringType())\
  ,T.StructField("meta_topic", T.StringType())\
  ,T.StructField("meta_partition", T.IntegerType())\
  ,T.StructField("meta_offset", T.IntegerType())\
  ,T.StructField("log_params_userid", T.IntegerType())\
                      ])

In [ ]:
spark.read.load("input",format="json",schema=dSchema).select("meta_dt","type").orderBy("meta_dt", ascending=False).show(15)

# Folder clean up

In [ ]:
%rm -f input/*.json

In [ ]:
%rm -rf *check*

In [ ]:
%rm -rf *output*

# Structured Streaming

In [8]:
#create Dataframe representing the stream of input lines from connection to localhost:9999
recentChange = spark.readStream.format("json").schema(dSchema).load("input")


In [9]:
recentChange.isStreaming

True

In [ ]:
rc_df = recentChange.withColumn("meta_dt", recentChange.meta_dt.cast(T.TimestampType()))
rc_df

In [11]:
partitioned_df = rc_df.withWatermark("meta_dt", "1 minute")\
  .writeStream\
  .format("parquet")\
  .option("path", "output")\
  .option("checkpointLocation", "checkpoint_dir")\
  .partitionBy("type")\
  .start()

In [16]:
partitioned_df.stop()

In [ ]:
# Assuming 'recentChange' is your streaming DataFrame
myquery = recentChange\
  .withColumn("meta_dt", recentChange.meta_dt.cast(T.TimestampType()))\
  .withWatermark("meta_dt", "1 minute")\
  .groupBy(F.window("meta_dt","1 minute")\
         , "type")\
  .count()\
  .writeStream\
  .format("json")\
  .outputMode("append")\
  .option("path", "output")\
  .option("checkpointLocation", "checkpoint_dir")\
  .start()



In [ ]:
# Assuming 'recentChange' is your streaming DataFrame; This cell is working without aggregation
# myquery = recentChange\
#   .withColumn("meta_dt", recentChange.meta_dt.cast(T.TimestampType()))\
#   .writeStream\
#   .format("csv")\
#   .outputMode("append")\
#   .option("path", "output")\
#   .option("header","true")\
#   .option("checkpointLocation", "checkpoint_dir")\
#   .start()

In [ ]:
%ls -lrt output

In [ ]:
myquery.stop()

In [ ]:
inmemoryQuery = recentChange\
  .withColumn("meta_dt", recentChange.meta_dt.cast(T.TimestampType()))\
  .groupBy(F.window("meta_dt","1 minute"), "type")\
  .count()\
  .writeStream\
  .queryName("myquery")\
  .format("memory")\
  .outputMode("complete")\
  .start()

In [ ]:
spark.sql("select * from myquery order by type").show(truncate=False)

In [ ]:
inmemoryQuery.stop()

In [ ]:
%ls -lr input

In [ ]:
table_query = rc_df.writeStream\
  .format("parquet")\
  .option("checkpointLocation", "checkpoint_dir_parquet")\
  .toTable("rc_table")

In [ ]:
spark.sql("select * from rc_table").show()

In [ ]:
table_query.stop()

In [ ]:
table_agg_query = rc_df\
  .withWatermark("meta_dt", "1 minute")\
  .groupBy(F.window("meta_dt","1 minute")\
         , "type")\
  .count()\
  .writeStream\
  .format("parquet")\
  .option("checkpointLocation", "checkpoint_dir_table_agg)")\
  .toTable("rc_table_agg")

In [ ]:
spark.sql("select window.start, type,count from rc_table_agg").show(truncate=False)

In [ ]:
table_agg_query.stop()

# Steps to normalize
- define schema with datatype as StringType
- read Json with the schema and select leaf nodes parent.child1.child2
- use this dataframe to cast and explode.

However, it is not dynamic and cannot handle unexpected nesting

In [ ]:
intensity_schema = T.StructType([\
   T.StructField("from", T.StringType())\
  ,T.StructField("to", T.StringType())\
  ,T.StructField("intensity", T.StructType([
      T.StructField("forecast", T.StringType())\
     ,T.StructField("actual", T.StringType())\
     ,T.StructField("index", T.StringType())]))\
                    ])

In [ ]:
intensity_df = spark.read.load(".",format="json",schema=intensity_schema).select("from","intensity.forecast")

In [ ]:
intensity_df.show(truncate=False)

+-----------------+--------+
|from             |forecast|
+-----------------+--------+
|2024-09-16T10:30Z|147     |
+-----------------+--------+



#Testing

In [ ]:
rate_df = spark.readStream.format("rate").option("rowsPerSecond", 100).load()

In [ ]:
rate_df.printSchema()

In [ ]:
rate_df.writeStream.format("memory").queryName("query_name").outputMode("append").start()

In [ ]:
spark.sql("select * from query_name").show(truncate=False)

In [ ]:
json_rate_df=rate_df.withWatermark("timestamp", "5 minutes")\
.groupBy(F.window("timestamp","1 minutes"))\
.count()\
.writeStream.format("json").option("path", "json_output").option("checkpointLocation", "checkpoint_dir_json")\
.trigger(processingTime="3 minutes")\
.start()

In [ ]:
datetime.datetime.strftime(datetime.datetime.utcnow(),"%Y-%m-%d %H:%M:%S")

'2024-09-11 15:05:24'

In [ ]:
%ls --full-time -lrt  json_output

In [ ]:
json_rate_df.stop()